# 패키지 불러오기

In [1]:
# 패키지 불러오기

import requests
import bs4
from bs4 import BeautifulSoup
import os
import random
import hashlib
import urllib.request
import urllib
from urllib.parse import urlparse, urlencode, parse_qs
import pandas as pd
import csv
import re

from datetime import datetime, timedelta
from urllib import parse
import time
import requests
import numpy as np
import json
from random import *
from tqdm.notebook import tqdm

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from fake_useragent import UserAgent

from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException

ua = UserAgent()
userAgent = ua.random
print(userAgent)

options = Options()
options.add_argument(f"user-agent={userAgent}")
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--proxy-server=socks5h://localhost:9050')

executable_path = '/home/book_crawling/geckodriver' # chrome driver의 경로
driver = webdriver.Firefox(executable_path=executable_path, options=options)

Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.3319.102 Safari/537.36


# 검색어 쿼리 생성 함수

In [2]:
def query_generate(orig_title):
    proc_sam = re.sub('[/(|\)|\[|\]|<\>『|』|\`|“|”|:|,|!|?|·|\"]', ' ', orig_title)
    proc_sampl = re.sub('\[.*\]', '', proc_sam)
    proc_sample = re.sub('\(.*\)', '', proc_sampl)
    proc_space = re.sub('\s+', ' ', proc_sample)
    arr = proc_space.split(' ')
    if len(arr) >= 5: 
        query = '+'.join(arr[:5])
    else:
        query = '+'.join(arr)
    if query[0] == '+':
        query = query[1:]
    if query[-1] == '+':
        query = query[:-1]
    return query, orig_title

# Ridi 책 정보 크롤링 함수

In [3]:
def ridi_crawl(title):
    
    book = {'제목' : title,
            '저자' : '없음',
            '출판사' : '없음',
            '총권수' : '없음',
            '완결여부' : '없음',
            '출간정보' : '없음',
            '파일정보' : '없음',
            'isbn' : '없음',
            '책소개' : '없음',
            '목차' : '없음',
            '별점' : '없음'
           }
    
    query = query_generate(title)[0]
    search_page = f'https://ridibooks.com/search/?q={query}'
    driver.get(search_page)
    driver.implicitly_wait(3)
    
    try:
        title = driver.find_element_by_tag_name('h3')
    except:
        return book
    
    link = title.find_element_by_tag_name('a')
    item_page = link.get_attribute('href')
    driver.get(item_page)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # 제목
    a = soup.select_one('meta[property="og:title"]')
    try:
        title = a['content']
    except:
        title = '없음'

    # 저자
    b = soup.select_one('.metadata.metadata_writer')
    try:
        author = b.text.replace('저', '').strip()
    except:
        author = '없음'

    # 출판사
    c = soup.select_one('.metadata.file_info.publisher_info')
    try:
        pub = c.text.replace('출판', '').strip()
    except:
        pub = '없음'

    # 총권수
    try:
        d = soup.select_one('.metadata_item.book_count')
        total = d.text.replace('출판', '').strip()
    except:
        total = '없음'

    # 완결여부
    try:
        e = soup.select_one('.metadata_item.complete')
        complete = e.text.strip()
    except:
        complete = '없음'

    # 출간정보
    f = soup.select_one('.Header_Metadata_Item.book_info.published_date_info')
    try:
        pub_info = f.ul.text.replace('\n', '').strip()
    except:
        pub_info = '없음'

    # 파일정보
    try:
        g = f.next_sibling
        try:
            file_info = g.text.replace('정보', '정보 ').replace('평균', ' 평균').replace('EPUB', 'EPUB ').replace('MB', 'MB ')
        except:
            file_info = '없음'
    except:
        file_info = '없음'

    # ISBN
    try:
        h = g.next_sibling
        try:
            isbn = h.text.replace('ISBN', '')
        except:
            isbn = '없음'
    except:
        isbn = '없음'

    # 책소개
    try:
        book_intro = soup.select_one('.introduce_paragraph').text
    except:
        book_intro = '없음'

    # 목차
    try:
        table = soup.select_one('.detail_box_module.detail_book_table').text
    except:
        table = '없음'
    
    # 별점
    try:
        star = soup.select_one('.StarRate_Score').text.replace('점', '')
    except:
        star = '없음'
        
    book = {'제목' : title,
            '저자' : author,
            '출판사' : pub,
            '총권수' : total,
            '완결여부' : complete,
            '출간정보' : pub_info,
            '파일정보' : file_info,
            'isbn' : isbn,
            '책소개' : book_intro,
            '목차' : table,
            '별점' : star
           }
        
    return book

# 샘플 데이터 크롤링

In [6]:
ridi_crawl(titles[55])

{'제목': '[코믹] 29세 독신은 이세계에서 자유롭게 살고…… 싶었다 1권',
 '저자': '오오하마이코           그림류토           원작',
 '출판사': '시프트코믹스',
 '총권수': '총 3권',
 '완결여부': '없음',
 '출간정보': '2020.06.04. 출간',
 '파일정보': '파일 정보  평균 125.9MB 184쪽',
 'isbn': '9791165800161',
 '책소개': '<[코믹] 29세 독신은 이세계에서 자유롭게 살고…… 싶었다> 29세 독신 미츠바 타이시는, 어느 날 갑자기 판타지 세계로!? \n어리둥절하면서도, 젊어진 육체와 치트 능력을 얻어 이 이세계를 자유롭게 살겠다!\n...라고 생각했으나…!? 거기서 만난, 미소녀 마르와의 알콩달콩하고, 말썽도 있는 이세계 생활이 시작된다!!\n\n*본 작품의 적정 열람 연령은 15세 이상입니다.',
 '목차': '\n\n목차\n\n\n프롤로그\n제1화\n제2화\n제3화\n제4화\n제5화\n펼쳐보기 \n\n',
 '별점': '3.9'}

# 대량 크롤링

input_csv_path : 크롤링할 서명 리스트 파일

output_csv_path : 크롤링 결과 csv 파일

In [5]:
input_csv_path = './0419_ridi.csv'
output_csv_path = './ridi_search_result.csv'

books_csv = pd.read_csv(input_csv_path, encoding='utf-8', error_bad_lines=False)
titles = books_csv.서명.tolist()

In [7]:
ridi_data = []
for title in tqdm(titles):
    time.sleep(2)
    result = ridi_crawl(title)
    ridi_data.append(result)
    
ridi_data_df = pd.DataFrame(ridi_data)
ridi_data_df.to_csv(output_csv_path, encoding='utf-8-sig')

  0%|          | 0/700 [00:00<?, ?it/s]